In [2]:
#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/13 16:36:17 WARN Utils: Your hostname, LAPTOP-F054CH70 resolves to a loopback address: 127.0.1.1; using 172.23.196.108 instead (on interface eth0)
22/09/13 16:36:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/13 16:36:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
curated_csv = spark.read.options(header = True).csv('../data/curated/full_data.csv')

In [3]:
curated_csv.limit(5)
print(curated_csv.count(),len(curated_csv.columns))

8151372 15


In [4]:
curated_csv2 = curated_csv.drop(curated_csv.address)
curated_csv3 = curated_csv2.drop(curated_csv.gender)
#curated_csv4 = curated_csv3.drop(curated_csv.user_name)
curated_csv3.limit(5)
print(curated_csv3.count(),len(curated_csv3.columns))

8151372 13


In [5]:
# check null value
curated_csv3 = curated_csv3.dropna()
print(curated_csv3.count(),len(curated_csv3.columns))

7817730 13


In [6]:
curated_csv3.limit(5)

user_id,merchant_abn,dollar_value,order_id,order_datetime,consumer_id,merchant_name,user_name,state,postcode,field,revenue_level,take_rate
13153,72472909171,112.17,baa29c79-6969-416...,2021-08-21,1018,Nullam Consulting,Kelly Frey,NSW,2058,digital goods: bo...,a,6.33
13153,23338656015,324.21,463dd933-d5b3-441...,2022-02-03,1018,Iaculis LLC,Kelly Frey,NSW,2058,"watch, clock, and...",b,3.25
13153,13093581573,117.23,9bd2915e-c3b5-47a...,2021-08-22,1018,Dis Parturient Co...,Kelly Frey,NSW,2058,"gift, card, novel...",e,0.24
13153,89726005175,24.57,cd1007fb-2336-44f...,2022-02-03,1018,Est Nunc Consulting,Kelly Frey,NSW,2058,tent and awning s...,a,6.01
13153,40590483301,26.08,a3e457c3-c0bb-467...,2021-08-15,1018,Donec Tempus Corp...,Kelly Frey,NSW,2058,"cable, satellite,...",c,3.1


In [7]:
# check wrong data
from os import curdir
from pyspark.sql import functions as F
new_data = curated_csv3.filter(
    (F.col('user_id') > 0 )
    & (F.col('dollar_value') >= 0)
    & (F.col('consumer_id') > 0)
    & (F.col('postcode') > 0)
)
print(new_data.count(),len(new_data.columns))

7817730 13


In [8]:
new_data.limit(5)

user_id,merchant_abn,dollar_value,order_id,order_datetime,consumer_id,merchant_name,user_name,state,postcode,field,revenue_level,take_rate
13153,72472909171,112.17,baa29c79-6969-416...,2021-08-21,1018,Nullam Consulting,Kelly Frey,NSW,2058,digital goods: bo...,a,6.33
13153,23338656015,324.21,463dd933-d5b3-441...,2022-02-03,1018,Iaculis LLC,Kelly Frey,NSW,2058,"watch, clock, and...",b,3.25
13153,13093581573,117.23,9bd2915e-c3b5-47a...,2021-08-22,1018,Dis Parturient Co...,Kelly Frey,NSW,2058,"gift, card, novel...",e,0.24
13153,89726005175,24.57,cd1007fb-2336-44f...,2022-02-03,1018,Est Nunc Consulting,Kelly Frey,NSW,2058,tent and awning s...,a,6.01
13153,40590483301,26.08,a3e457c3-c0bb-467...,2021-08-15,1018,Donec Tempus Corp...,Kelly Frey,NSW,2058,"cable, satellite,...",c,3.1


In [9]:
# check repetation 
new_data1 = new_data.dropDuplicates(['order_id'])
print(new_data1.count(),len(new_data1.columns))

7817730 13


In [10]:
# check the order datetime
from pyspark.sql import functions as F
new_df = new_data1.filter(
    (F.col('order_datetime') >= '2021-02-28') 
    & (F.col('order_datetime') <= '2022-08-28')
)

print(new_df.count(),len(new_df.columns))

7817730 13


In [11]:
'''
# check postcode
new_df = new_df.withColumn("length_of_str",F.length('postcode'))
new_df.select('length_of_str').describe()
new_df = new_df.filter(
    (F.col('length_of_str') <= 4)
    & (F.col('length_of_str') >=3)
)
new_df = new_df.drop('length_of_str')
print(new_df.count(),len(new_df.columns))


new_df.select('postcode').describe()


# check postcode
new_df = new_df.withColumn("length_of_str",F.length('postcode'))
new_df.select('length_of_str').describe()
new_df = new_df.filter(
    (F.col('length_of_str') == 4)
    #&(F.col('length_of_str') >=3 )
)
new_df = new_df.drop('length_of_str')
print(new_df.count(),len(new_df.columns))
'''

'\n# check postcode\nnew_df = new_df.withColumn("length_of_str",F.length(\'postcode\'))\nnew_df.select(\'length_of_str\').describe()\nnew_df = new_df.filter(\n    (F.col(\'length_of_str\') <= 4)\n    & (F.col(\'length_of_str\') >=3)\n)\nnew_df = new_df.drop(\'length_of_str\')\nprint(new_df.count(),len(new_df.columns))\n\n\nnew_df.select(\'postcode\').describe()\n\n\n# check postcode\nnew_df = new_df.withColumn("length_of_str",F.length(\'postcode\'))\nnew_df.select(\'length_of_str\').describe()\nnew_df = new_df.filter(\n    (F.col(\'length_of_str\') == 4)\n    #&(F.col(\'length_of_str\') >=3 )\n)\nnew_df = new_df.drop(\'length_of_str\')\nprint(new_df.count(),len(new_df.columns))\n'

In [4]:
df.createOrReplaceTempView("EMP")

# PySpark SQL
sql_str="select state, count(*) as count from EMP " + \
"group by state"
spark.sql(sql_str).show()

+-----+-----+
|state|count|
+-----+-----+
|   NY|    5|
|   CA|    4|
+-----+-----+



In [ ]:
# check state
cur_df = new_df.filter(
    (F.col('state') == 'NT')
    | (F.col('state') == 'NSW')
    | (F.col('state') == 'QLD')
    | (F.col('state') == 'SA')
    | (F.col('state') == 'TAS')
    | (F.col('state') == 'VIC')
    | (F.col('state') == 'WA')
    | (F.col('state') == 'ACT')
)
print(cur_df.count(),len(cur_df.columns))


11873770 12
